In [1]:
import pandas as pd
import numpy as np
import pymysql


In [2]:
conn = pymysql.connect(host='localhost', user='root',
                                 password='',db='second', port=3306, autocommit=False, charset='utf8')
sql = '''
    select 
    工作职称, 
    发布时间, 
    待遇, 
    薪资, 
    学历
    from job5
'''
df = pd.read_sql(sql,conn)#参数：查询语句+连接配置
df.head()

,工作职称,发布时间,待遇,薪资,学历
0,大数据运维工程师,05-10发布,统计数据平台,7千-1万·13薪,北京·朝阳区|无需经验|本科
1,大数据分析师（实习生）,05-26发布,互联网,7-9千,兰州|1年|大专
2,甘肃-IT-大客户销售经理/总监（政府、企业、金融）,04-17发布,计算机,1.5-3万,兰州|1年|本科
3,WW-大客户销售经理（甘肃）,05-26发布,数据分析,1-1.5万,兰州·城关区|5-7年|大专
4,大客户销售代表,05-25发布,五险一金,1-2万,武汉·江岸区|1年|大专


In [3]:
# 查看是否有缺失值
df.isnull().sum()

工作职称    0
发布时间    0
待遇      0
薪资      0
学历      0
dtype: int64

In [4]:
print(df[df['薪资'].isnull()])

Empty DataFrame
Columns: [工作职称, 发布时间, 待遇, 薪资, 学历]
Index: []


In [5]:
data = df

In [6]:
# 将相关信息分组，找到对应的信息，存入列表
diqu = []
jingyan = []
xueli = []
for i in data['学历']:
    s = i.split("|")
    diqu.append(s[0])
    jingyan.append(s[1])
    xueli.append(s[2])

In [7]:
# 删除旧的列
data.drop(['学历'],axis=1,inplace=True)

In [8]:
data.head()

,工作职称,发布时间,待遇,薪资
0,大数据运维工程师,05-10发布,统计数据平台,7千-1万·13薪
1,大数据分析师（实习生）,05-26发布,互联网,7-9千
2,甘肃-IT-大客户销售经理/总监（政府、企业、金融）,04-17发布,计算机,1.5-3万
3,WW-大客户销售经理（甘肃）,05-26发布,数据分析,1-1.5万
4,大客户销售代表,05-25发布,五险一金,1-2万


In [9]:
# 为原数据添加新的列
data.insert(4,"学历",xueli,True)
data.insert(5,"经验",jingyan,True)
data.insert(6,"地区",diqu,True)

In [10]:
# 查看是否有缺失值
data.isnull().sum()

工作职称    0
发布时间    0
待遇      0
薪资      0
学历      0
经验      0
地区      0
dtype: int64

In [11]:
data1 = data

In [12]:
dict = {}
for n in set(data1.薪资):
    list(data1.薪资).count(n)
    dict[n] = list(data1.薪资).count(n)
dict

{'0.8-1万': 7,
 '9千-1.3万': 7,
 '5千-1万': 7,
 '1.5-3万': 7,
 '3.5-5千': 7,
 '8千-1.2万·13薪': 7,
 '5-8千': 7,
 '9千-1.5万': 7,
 '2-4万': 7,
 '7-9千': 7,
 '1-2万': 7,
 '15-25万/年': 7,
 '9千-1.2万': 7,
 '1-1.5万': 7,
 '1-2万·13薪': 7,
 '6-8千': 7,
 '8千-1万': 7,
 '7千-1万·13薪': 7,
 '1.2-2.4万': 7}

In [13]:
data3 = data1['薪资']
data3

0      7千-1万·13薪
1           7-9千
2         1.5-3万
3         1-1.5万
4           1-2万
         ...    
128         6-8千
129         5-8千
130         2-4万
131       3.5-5千
132        5千-1万
Name: 薪资, Length: 133, dtype: object

In [14]:
df = []
data3 = data['薪资']
for j in data3:
    if j == "":
        df.append("16000")
    if "千-" in j or "万-" in j:
        df.append(j)
    elif "及以下" in j:
        s = j.replace("及以下","")
        df.append(s)
    elif '-' in j:
        if int(j[0]) > 5:
            s = j.replace("-","千-")
        else:
            s = j.replace("-", "万-")
        df.append(s)
print(len(df))

133


In [15]:
salarys = []
for row in df:
    sal = ''
    print("-------------")
    print(row)
    for s in row:
        if s == '千':
            sal += '*1000'
        elif s == '万':
            sal += '*10000'
        elif s == '薪':
            s += ""
        elif s == 13:
            s += ""
        elif s == '千及以下':
            s += "千"
        elif s == '·':
            s += ""
        elif s == '年':
            s += ""
        elif s == '/':
            s += ""
        elif s == '-':
            sal += '+'
        else:
            sal += s
    print(sal)


    if row == "16000":
        salarys.append(16000)
    else:
        d = sal.split("·")
        z = d[0]
        print("运算式", z)
        ave = eval(sal.split("·")[0]) / 2
        if ave > 30000:
            ave = ave / 12
        print("最终值", ave)
        salarys.append(ave)
print(len(salarys))

-------------
7千-1万·13薪
7*1000+1*1000013
运算式 7*1000+1*1000013
最终值 41958.875
-------------
7千-9千
7*1000+9*1000
运算式 7*1000+9*1000
最终值 8000.0
-------------
1.5万-3万
1.5*10000+3*10000
运算式 1.5*10000+3*10000
最终值 22500.0
-------------
1万-1.5万
1*10000+1.5*10000
运算式 1*10000+1.5*10000
最终值 12500.0
-------------
1万-2万
1*10000+2*10000
运算式 1*10000+2*10000
最终值 15000.0
-------------
1.2万-2.4万
1.2*10000+2.4*10000
运算式 1.2*10000+2.4*10000
最终值 18000.0
-------------
0.8万-1万
0.8*10000+1*10000
运算式 0.8*10000+1*10000
最终值 9000.0
-------------
15万-25万/年
15*10000+25*10000
运算式 15*10000+25*10000
最终值 16666.666666666668
-------------
8千-1万
8*1000+1*10000
运算式 8*1000+1*10000
最终值 9000.0
-------------
1万-2万·13薪
1*10000+2*1000013
运算式 1*10000+2*1000013
最终值 83751.08333333333
-------------
9千-1.2万
9*1000+1.2*10000
运算式 9*1000+1.2*10000
最终值 10500.0
-------------
8千-1.2万·13薪
8*1000+1.2*1000013
运算式 8*1000+1.2*1000013
最终值 50333.98333333333
-------------
9千-1.3万
9*1000+1.3*10000
运算式 9*1000+1.3*10000
最终值 11000.0
-------------
9千-1.5

In [16]:
salarys = [int(i) for i in salarys]

In [17]:
data1.薪资=salarys

In [19]:
data1

,工作职称,发布时间,待遇,薪资,学历,经验,地区
0,大数据运维工程师,05-10发布,统计数据平台,41958,本科,无需经验,北京·朝阳区
1,大数据分析师（实习生）,05-26发布,互联网,8000,大专,1年,兰州
2,甘肃-IT-大客户销售经理/总监（政府、企业、金融）,04-17发布,计算机,22500,本科,1年,兰州
3,WW-大客户销售经理（甘肃）,05-26发布,数据分析,12500,大专,5-7年,兰州·城关区
4,大客户销售代表,05-25发布,五险一金,15000,大专,1年,武汉·江岸区
...,...,...,...,...,...,...,...
128,数据统计专员,05-26发布,数据仓库,7000,大专,无需经验,兰州
129,初级数据实施工程师（JR11dy）(J10526),05-23发布,oracle,29000,本科,1年,兰州
130,海外财务经理G00955,05-12发布,etl,30000,本科,5-7年,广州·天河区
131,商务,05-26发布,统计,20000,本科,2年,兰州·城关区


In [20]:
a_list = []
for i in data1['经验']:
    if i == "无需经验" or i == "":
        i= 0
    else:
        i = i[0:1]
    a_list.append(int(i))

In [21]:
data1['经验'] = a_list

In [23]:
data1

,工作职称,发布时间,待遇,薪资,学历,经验,地区
0,大数据运维工程师,05-10发布,统计数据平台,41958,本科,0,北京·朝阳区
1,大数据分析师（实习生）,05-26发布,互联网,8000,大专,1,兰州
2,甘肃-IT-大客户销售经理/总监（政府、企业、金融）,04-17发布,计算机,22500,本科,1,兰州
3,WW-大客户销售经理（甘肃）,05-26发布,数据分析,12500,大专,5,兰州·城关区
4,大客户销售代表,05-25发布,五险一金,15000,大专,1,武汉·江岸区
...,...,...,...,...,...,...,...
128,数据统计专员,05-26发布,数据仓库,7000,大专,0,兰州
129,初级数据实施工程师（JR11dy）(J10526),05-23发布,oracle,29000,本科,1,兰州
130,海外财务经理G00955,05-12发布,etl,30000,本科,5,广州·天河区
131,商务,05-26发布,统计,20000,本科,2,兰州·城关区


In [22]:
data1.to_csv("last.csv",encoding="UTF-8")